In [102]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohang62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [104]:
print(len(train.index))
print(len(test.index))

5000
1800


In [105]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [106]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i != "@USER":
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)

In [107]:
train.head()

,label,response
0,SARCASM,get .. obviously care would've moved right alo...
0,SARCASM,trying protest talking labels label wtf make em
0,SARCASM,makes insane money movies einstein #learnhowth...
0,SARCASM,meanwhile trump even release sat scores wharto...
0,SARCASM,pretty sure anti-lincoln crowd claimed democra...


In [108]:
with open('train_ft.txt', 'w') as f:
    for each_text, each_label in zip(train['response'], train['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [133]:
import fasttext
model = fasttext.train_supervised('train_ft.txt', epoch=300, lr=0.05, wordNgrams=5, verbose=2, minCount=1)

In [134]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{precision}')
    print(f'{recall}')

print_results(*model.test('train_ft.txt'))

1.0
1.0


In [135]:
def predict_is_sarcastic(text):
    return model.predict(text, k=2)

In [136]:
!rm answer.txt

In [137]:
with open('answer.txt', 'w') as f:
    for each_id, each_text in zip(test['id'], test['response']):
        labels, probs = predict_is_sarcastic(f'{each_text}')
        l = None
        if (probs[0] > probs[1] and labels[0] == "__label__SARCASM"):
            l = "SARCASM"
        elif (probs[0] > probs[1] and labels[0] != "__label__SARCASM"): 
            l = "NOT_SARCASM"
        elif (probs[0] < probs[1] and labels[0] == "__label__SARCASM"): 
            l = "NOT_SARCASM"
        else:
            l = "SARCASM"
        f.writelines(f'{each_id},{l}\n')